<font color="#CA3532"><h1 align="left">**Aprendizaje por Refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed Bandits Contextuales**</h2></font>

En este notebook se considera el siguiente problema:

Tenemos un conjunto de clientes, y de cada cliente tenemos dos datos:

* **Edad**
* **Ingresos generados** hasta el momento por ese cliente (gasto de ese cliente en nuestra empresa).

Nuestra compañía ofrece tres promociones, y el problema es encontrar qué promoción ofrecemos a cada cliente. Una vez que se ofrece una promoción a un cliente, este puede ignorarla o acogerse a ella, generando en este caso una ganancia para la compañía. Por tanto, las **acciones** posibles son las promociones que le podemos ofrecemos al cliente (tres en este ejemplo).

En caso de que un cliente se acoja a una promoción la ganancia para la compañía será:

* **Promoción 0**: ganancia=10
* **Promoción 1**: ganancia=25
* **Promoción 2**: ganancia=100


Hasta aquí son datos que la compañía tiene bajo control.
Sin embargo la compañía desconoce cómo van a responder los clientes a las promociones. Es decir, desconoce la probabilidad de que un cliente $x_i$ se acoja a la promoción $j$ en caso de que se le ofrezca. Es decir, se desconocen las siguientes cantidades:

* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,0)$
* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,1)$
* $p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,2)$


y la decisión óptima sería, una vez calculadas estas tres cantidades para un cliente, elegir la promoción que conlleve mayor ganancia esperada.

Si supiéramos el valor de estas cantidades para cada cliente (se pueden ver como funciones que dependen del cliente y la promo ofrecida), se podría calcular la ganancia esperada de cada acción (promo ofrecida) como:

* Si se ofrece promo 0:
$$
Q\,teórica(promo \,0, x_i) = 10 \cdot 
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,0)
$$
* Si se ofrece promo 1:
$$
Q\,teórica(promo \,1, x_i) = 25 \cdot 
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,1)
$$
* Si se ofrece promo 2:
$$
Q\,teórica(promo \,2, x_i) = 100 \cdot 
p(acoge\,promo \,\vert\, x_i, le \, ofrecemos \, promo \,2)
$$


Sin embargo, **es imposible** conocer a priori esas cantidades.
La compañía espera que cada cliente responda de una manera diferente a las promociones de acuerdo a sus características, pero la compañía desconoce los segmentos de clientes y cómo responden a las promociones.

**El enorme valor del modelo contextual bandit** está en que dicha técnica va a construir esos segmentos y las funciones que modelan la respuesta de cada cliente a cada promo. Lo va a hacer de manera **online** (incremental, según se van ofreciendo promos a clientes) y de tal forma que va a intentar maximizar la ganancia obtenida en todo el proceso (incluyendo los pasos iniciales de recolección de información).

In [ ]:
COLAB = True

In [ ]:
!ls

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1tcbutY0wW-JWodaVSTuAhxn4pqPVyqZX',
                                        dest_path='./simula_clientes_problema1.py') # simulamos los clientes
    gdd.download_file_from_google_drive(file_id='1fCnGzS5U_x-k_03op_XJkHVS4jpvjSxS',
                                        dest_path='./spacebandits.zip', unzip=True) # el algoritmo de aprendizaje por refuerzo

In [ ]:
import numpy as np
import pandas as pd
from random import random, randint
import matplotlib.pyplot as plt
from tqdm import tqdm

%config InlineBackend.figure_format='retina'
%matplotlib inline

In [ ]:
from simula_clientes_problema1 import cliente

In [ ]:
c = cliente() # x es una instancia de la clase "cliente", que tengo implementada en simula_clientes_problema1

In [ ]:
# edad, ingresos generados en la compañía hasta ahora
c.get_context() # mi simulador me devuelve un array con las características del cliente

In [ ]:
accion = 0
c.get_reward(accion) # me devuelve el reward que obtengo por la acción que he tomado (si es 0, el cliente no ha comprado nada)

### **Estrategia 1: recomendación al azar**

In [ ]:
N = 10000 # número de clientes que voy a simular
reward_total = 0
for _ in range(N):
  c = cliente() # genero un nuevo cliente
  accion = np.random.choice(3) # tomo una acción aleatoria
  reward_total += c.get_reward(accion) # obtengo el reward que me da esa acción (si lo compra o no)

print("Reward promedio:", reward_total/N) 

### **Estrategia 2: Multiarmed Bandit contextual**

In [ ]:
from space_bandits import LinearBandits

n_acciones = 3 # número de promociones diferentes
n_features = 2 # número de variables de contexto (edad e ingresos generados)

N = 10000

agente = LinearBandits(n_acciones, n_features, initial_pulls=100) # initial_pulls: duración de la fase de exploración pura (los primeros 100 clientes reciben una promoción aleatoria por no tener un dataset al principio)
reward_promedio = []
reward_total = 0
for i in tqdm(range(N)):
  c = cliente() # cliente con el que contacto
  contexto = c.get_context()
  accion = agente.action(contexto) # choose_action: elegir la acción que voy a tomar con el cliente
  reward = c.get_reward(accion) # reward por haber comprado o no
  agente.update(contexto, accion, reward) # actualización del modelo para seguir aprendiendo

  reward_total += reward
  reward_promedio.append(reward_total/(i+1)) # i+1 es el número de iteraciones

print("Reward promedio:", reward_total/N)
plt.plot(reward_promedio)
plt.title("Evolución del reward promedio");

Con esa estragia 2 hemos doplado la ganancia esperada de la estrategia 1.

**¿Cómo evalúa el modelo cada una de las acciones?**

In [ ]:
c = cliente()

In [ ]:
contexto = c.get_context()
contexto

In [ ]:
agente.action(contexto) # nos devuele el producto recomendado para ese cliente (0, 1 o 2) con el Q-valor más alto

In [ ]:
agente.expected_values(contexto) # Q(s,a) por cada acción a, y para el cliente s (s=contexto)

In [ ]:
# Parámetros que aprende el agente:
# 3 arrays (porque hay 3 acciones posibles, 3 productos)
# En cada array el último elemento es el término constante de un modelo lineal
# Los elementos primero y segundo son los coeficientes de edad e ingresos respectivamente
agente.mu

In [ ]:
# Separo los parámetros del contexto en edad e ingresos
edad, ingresos_generados = contexto
edad, ingresos_generados

In [ ]:
agente.mu[0][-1] + agente.mu[0][0]*edad + agente.mu[0][1]*ingresos_generados # Q(s,a) para el producto 0

In [ ]:
agente.mu[1][-1] + agente.mu[1][0]*edad + agente.mu[1][1]*ingresos_generados # Q(s,a) para el producto 1

In [ ]:
agente.mu[2][-1] + agente.mu[2][0]*edad + agente.mu[2][1]*ingresos_generados # Q(s,a) para el producto 2